## Notebook settings

In [1]:
%load_ext autoreload
%autoreload 2

%autosave 10

#%load_ext lab_black

Autosaving every 10 seconds


In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join("..")))

## Imports

In [3]:
from lfp_analysis.data import *

In [4]:
import numpy as np
import pandas as pd
import h5py
from librosa.util import frame

from pathlib import Path

import matplotlib.pyplot as plt

%matplotlib widget

# Label processor:

In [5]:
def make_df(label, win_len_sec, perc_train=0.8):
    win_len_samp = int(win_len_sec * 2048)
    n_win = len(label) // win_len_samp

    # Window idx:
    id_l = np.array(range(n_win)) * win_len_samp
    data_df = pd.DataFrame(
        np.stack((id_l, id_l + win_len_samp)).T, columns=["id_start", "id_end"]
    )

    # Train / Valid split
    data_df["is_valid"] = 0
    data_df.iloc[int(n_win * perc_train) :, 2] = 1

    # Get label for each window:
    # Label as "movement" if any sample inside window is individually labeled as movement:
    data_df["label"] = np.any(frame(label, win_len_samp, win_len_samp), axis=0)

    return data_df

# Import data:

In [6]:
DATA_PATH = Path("./../data")
PAT_ID = Path('ET2')
DATASET_NAME = Path('Pouring_off')

H5_FILE = DATA_PATH / 'preprocessed' / PAT_ID / DATASET_NAME.with_suffix('.h5')


In [7]:
h5 = h5py.File(H5_FILE, "r")
label = h5["label"][:]
h5.close()

In [8]:
label.shape

(627750,)

In [9]:
df = make_df(label, 1, perc_train=0.8)

In [10]:
df.is_valid.value_counts()

0    244
1     62
Name: is_valid, dtype: int64

In [11]:
df

,id_start,id_end,is_valid,label
0,0,2048,0,False
1,2048,4096,0,False
2,4096,6144,0,False
3,6144,8192,0,False
4,8192,10240,0,False
...,...,...,...,...
301,616448,618496,1,False
302,618496,620544,1,False
303,620544,622592,1,False
304,622592,624640,1,False


In [12]:
fig,ax = plt.subplots()
ax.plot(df.label)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Save to file:

In [13]:
TARGET_PATH = DATA_PATH / "TF_and_df" / PAT_ID / DATASET_NAME.with_suffix('.csv')

In [14]:
df.to_csv(TARGET_PATH)